# XGBoost Stock Forecasting

In [1]:
%pip install xgboost --no-binary xgboost -v

Using pip 23.1.2 from /Users/nhtrinh/Desktop/cmpe255-team-project/venv/lib/python3.11/site-packages/pip (python 3.11)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.0 MB/s eta 0:00:00 MB/s eta 0:00:0101
  Installing build dependencies ...   Running command pip subprocess to install build dependencies
                                                0.0/75.8 kB ? eta -:--:--
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 3.3 MB/s eta 0:00:00
    Using cached packaging-24.2-py3-none-any.whl (65 kB)
    Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)

  [notice] A new release of pip is available: 23.1.2 -> 24.3.1
  [notice] To update, run: pip install --upgrade pip
done
  Getting requirements to build wheel ...   Running command Getting requirements to build wheel
done
  Preparing metadata (pyproject.toml) ...   Running command Preparing metadata (pyproject.toml)
  INFO:xgboost.packager.build_wheel:Parsed build configuration: {'hide_cxx_symbols': True, 'use

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import asarray
from pandas import DataFrame, concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
DATA_FILENAME = "weekly_adjusted_SPY.csv"
STOCK_SYMBOL = "SPY"

In [ ]:
def load_dataset() -> pd.DataFrame:
    # Load the dataset
    data = pd.read_csv(f"data/{DATA_FILENAME}")
    data = data[['timestamp', 'adjusted close', 'volume']]
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data.sort_values('timestamp', inplace=True)

    # Prepare the data
    data['adjusted close'] = data['adjusted close'].astype(float)
    data['volume'] = data['volume'].astype(float)
    series = data[['adjusted close', 'volume']].values

    return series

In [ ]:

# Transform a time series dataset into supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    df = DataFrame(data)
    cols = list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    agg = concat(cols, axis=1)
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

In [ ]:

# Split dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test, :], data[-n_test:, :]

In [ ]:
def xgboost_forecast(train, testX):
    # XGBoost model
    train = asarray(train)
    trainX, trainy = train[:, :-1], train[:, -1]
    model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
    model.fit(trainX, trainy)
    yhat = model.predict(asarray([testX]))
    return yhat[0]


def walk_forward_validation(data, n_test):
    # Walk-forward validation
    predictions = list()
    train, test = train_test_split(data, n_test)
    history = [x for x in train]
    for i in range(len(test)):
        testX, test_y = test[i, :-1], test[i, -1]
        yhat = xgboost_forecast(history, testX)
        predictions.append(yhat)
        history.append(test[i])
    return test[:, -1], predictions



# Prepare data
series = load_dataset()
n_in = 6
supervised = series_to_supervised(series, n_in=n_in)
n_test = 50

# Evaluate
y, yhat = walk_forward_validation(supervised, n_test)

In [ ]:
# Plot the results
plt.title(
    f'Stock Forecast vs Actual for {STOCK_SYMBOL} - '
    + pd.Timestamp.now().strftime("%Y-%m-%d")
)
plt.plot(y, label='Expected')
plt.plot(yhat, label='Predicted')
plt.legend()
# If FigureCanvasAgg is interactive show plot
if matplotlib.is_interactive():
    plt.show()
else:
    plt.savefig(f'output/XGBoost_{STOCK_SYMBOL}_forecast.png')